In [1]:
print("Hello World")

Hello World


In [2]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from io import StringIO
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
tab=widgets.Tab()
placeholder=widgets.Label()
tab.children=[placeholder,placeholder,placeholder]
tab.set_title(0,"Upload")
tab.set_title(1,"Describer")
tab.set_title(2,"Plotter")
tab

In [4]:
up =widgets.FileUpload(
    accept="", #Dosya türü
    multiple=False #Birden fazla dosyal yüklenebilir
)
up

FileUpload(value=(), description='Upload')

In [5]:
#output widgets çıktı ekranı
out=widgets.Output(layout=({"border":"1px solid black"})) #çıktı ekranında siyah çizgi
tab.children=[up,out,out]
tab

In [6]:
#selectmultiple widgets çoklu seçim
eraser=widgets.SelectMultiple(
    options=["tab",'"'], #seçekler
    disabled=False,  #etkin yapar
    description="Eraser", #tanım
    value=["tab"] #seçili değer
) 
eraser

SelectMultiple(description='Eraser', index=(0,), options=('tab', '"'), value=('tab',))

In [7]:
#radiobutton widgets
delim=widgets.RadioButtons(
    options=[";",",",""],
    disabled=False,
    description="Seperator",

)
delim

RadioButtons(description='Seperator', options=(';', ',', ''), value=';')

In [8]:
#intslider kaydırma
rows=widgets.IntSlider(
    value=0,
    step=1,
    description="of lines:",
    disabled=False,
    orientation="horizontal", # Kaydırma çubuğunun yatay ("horizontal") veya dikey ("vertical") 
    readout=True, #çubuğun değerinin gösterilmesi
    readout_format="d" #tam sayı formatı
)
rows

IntSlider(value=0, description='of lines:')

In [9]:
#accordion widgets
accordion=widgets.Accordion()
accordion.children=[up,delim,rows]
accordion.set_title(0,"File Selection")
accordion.set_title(1,"Delimeter")
accordion.set_title(2,"Skip Rows")
accordion

Accordion(children=(FileUpload(value=(), description='Upload'), RadioButtons(description='Seperator', options=…

In [10]:
#button widgets
button_upload=widgets.Button(
    enabled=False,
    button_style="warning", #buton türü info,danger,success
    icon="warning", #sembol
    tooltip="Clic to Upload", #üstüne geline yazan
    description="Upload" #tanım
)
button_preview=widgets.Button(
    description="Preview",
    enabled=False,
    button_style="info",
    tooltip="Clic to Preview",
    icon="search"
)
button_upload
button_preview

Button(button_style='info', description='Preview', icon='search', style=ButtonStyle(), tooltip='Clic to Previe…

In [11]:
button_plot=widgets.Button(
    disabled=False,
    button_style="danger",
    icon="pencil",
    description="Plot",
    tooltip="Click to Plot"
)
button_plot

Button(button_style='danger', description='Plot', icon='pencil', style=ButtonStyle(), tooltip='Click to Plot')

In [12]:
#groupping HBox yatay - Vbox dikey gruplama 
vb=widgets.VBox([eraser,delim])
vb

In [13]:
hb=widgets.HBox([delim,eraser])
hb

In [14]:
accordion.children=[
    up,
    widgets.VBox([delim,eraser]),
    rows]

accordion_box=widgets.VBox([accordion,widgets.HBox([button_upload,button_preview]),out])
accordion_box


In [15]:
#toggle buttons bir buton çalışır diğerleri deactive olur(radiobutton misali)
toggle=widgets.ToggleButtons(
    options=["Preview ","Info ","Stats "],
    description="Options:",
    disabled=False,
    button_style="warning",
    icons=["search","info","tachometer"]
)
toggle


ToggleButtons(button_style='warning', description='Options:', icons=('search', 'info', 'tachometer'), options=…

In [16]:
#dropwdown widgets
x_axis=widgets.Dropdown(
    options=[""],
    value="",
    description="X-Axis",
    disabled=False
)
y_axis = widgets.Dropdown(
    options=[""],
    value="",
    description="Y-Axis",
    disabled=False
)


graph_type=widgets.Dropdown(
    options=["Bar Chart","Line Chart"],
    description="Chart Type:",
    disabled=False,
    value="Bar Chart"
)
graph_type

Dropdown(description='Chart Type:', options=('Bar Chart', 'Line Chart'), value='Bar Chart')

In [17]:
#color picker 
color_picker=widgets.ColorPicker(
    concise=False,# Renk seçimi modunu belirleyen bir boolean değer. False olarak ayarlanırsa, kullanıcı renk seçmek için bir renk paleti görecektir. True olarak ayarlanırsa, kullanıcı renk seçmek için bir renk kodunu (RGB veya HEX) yazabilir.x
    description="Color Picker",
    value="red",
    disabled=False
)
color_picker


ColorPicker(value='red', description='Color Picker')

In [18]:
children= [
    accordion_box,
    widgets.VBox([toggle,out]),
    widgets.HBox([
        widgets.HBox([graph_type,color_picker]),
        widgets.HBox([x_axis,y_axis]),
        button_plot,
        out
    ])]
tab.children=children
tab

In [19]:
#Value import #with ifadesi kullanıldığında, belirli bir nesnenin belirli bir eylem veya işlem süresi boyunca otomatik olarak başlatılmasını ve kapatılmasını sağlar
def content_parser():
    if up.value=={} :
        with out:
            print("No CSV loaded")
    else:
        typ, content= "",""
        up_value=up.value
        for i in range(len(up_value)):
            typ=up_value[i]["metadata"]["type"]   # up_value sözlüğündeki i anahtarının altındaki "metadata" anahtarının altındaki "type" anahtarının değerine atanır    

        if typ=="text/csv":
            content=up_value[i]["content"]
            content_str=str(content,"utf-8")  #i anahtarının altındaki content değeri alınıp string türkçe karakter olararak dönüştü.içerik türkçe karakter oldu

            if eraser.value != {}:
                for val in eraser.value:
                    if val=="tab":
                        content_str=content_str.replace("\t","")
                    else:
                        content_str=content_str.replace(val,"")    
            if content_str!="":                                 #StringIO sınıfını kullanarak bir StringIO nesnesi oluşturur. StringIO, bir karakter dizisini bellekteki dosya gibi işleme imkanı sağlar.
                str_io=StringIO(content_str)
                return str_io          

def df_converter():
    content=content_parser()
    if content is not None:
        df=pd.read_csv(content, sep=delim.value , index_col=False ,skiprows=rows.value)
        return df   
    else:
        return None    #content_parser fonksiyonu ile alınan bir CSV içeriğini okuyarak, belirtilen ayarlara göre bir pandas DataFrame'ine dönüştürüyor


In [20]:
tab

In [21]:
df_converter()

In [22]:
def preview():
    df=df_converter()
    with out:
        out.clear_output()
        print("\n----Now this is how your DF looks like----\n  ")
        if df is not None:
            print(df.head(10))
        else:
            print("Configuration is wrong/missing....")        

In [23]:
def preview_clicked(b):
    preview()

button_preview.on_click(preview_clicked)

In [25]:
def upload_click():
    df=df_converter()
    with out:
        out.clear_output()
        print("\n----Now this is how your DF looks like----\n  ")
        if df is not None:
            print(df)
            x_axis.options=df.columns
            y_axis.options=df.columns
        else:
            print("Configuration is wrong/missing....")    


In [26]:
def desc():                #Eğer info_level "Info" ise, df.info(verbose=True) ifadesiyle veri çerçevesinin bilgilerini görüntüler. Bu ifade, df veri çerçevesinin detaylı bilgilerini (info() metoduyla) çıktı olarak görüntüler ve verbose=True parametresiyle ayrıntılı bir çıktı elde eder.
    info_level=toggle.value
    if info_level!={}:
        df=df_converter()
        with out:
            out.clear_output()
            print(info_level)
        if df is not None:
            if info_level == "Info":
                print(df.info(verbose=True))    
            elif info_level =="Stats":
                print(df.describe()) 
            elif info_level=="Preview":
                print(df.head(5))

        else:
            print("Configuration is wrong/missing....")                     